# SGLang Frontend Language

SGLang frontend language can be used to define simple and easy prompts in a convenient, structured way.

## Launch A Server

Launch the server in your terminal and wait for it to initialize.

In [1]:
import requests
import os

from sglang import assistant_begin, assistant_end
from sglang import assistant, function, gen, system, user
from sglang import image
from sglang import RuntimeEndpoint, set_default_backend
from sglang.srt.utils import load_image
from sglang.test.test_utils import is_in_ci
from sglang.utils import print_highlight, terminate_process, wait_for_server

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd


server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path Qwen/Qwen2.5-7B-Instruct --host 0.0.0.0"
)

wait_for_server(f"http://localhost:{port}")
print(f"Server started on http://localhost:{port}")

[2025-07-19 00:42:00] server_args=ServerArgs(model_path='Qwen/Qwen2.5-7B-Instruct', tokenizer_path='Qwen/Qwen2.5-7B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Qwen/Qwen2.5-7B-Instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, impl='auto', host='0.0.0.0', port=37655, nccl_port=None, mem_fraction_static=0.874, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=765302064, constrai

[2025-07-19 00:42:13] Attention backend not explicitly specified. Use fa3 backend by default.
[2025-07-19 00:42:13] Init torch distributed begin.


[2025-07-19 00:42:19] Init torch distributed ends. mem usage=0.00 GB


[2025-07-19 00:42:20] Load weight begin. avail mem=78.50 GB


[2025-07-19 00:42:20] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


...

[2025-07-19 00:42:24] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=32768, available_gpu_mem=62.81 GB


[2025-07-19 00:42:24] INFO:     Started server process [55668]
[2025-07-19 00:42:24] INFO:     Waiting for application startup.
[2025-07-19 00:42:24] INFO:     Application startup complete.
[2025-07-19 00:42:24] INFO:     Uvicorn running on http://0.0.0.0:37655 (Press CTRL+C to quit)


[2025-07-19 00:42:25] INFO:     127.0.0.1:58568 - "GET /v1/models HTTP/1.1" 200 OK
[2025-07-19 00:42:25] INFO:     127.0.0.1:58578 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-19 00:42:25] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-19T00:42:25.604758


[2025-07-19 00:42:26] INFO:     127.0.0.1:58590 - "POST /generate HTTP/1.1" 200 OK
[2025-07-19 00:42:26] The server is fired up and ready to roll!


Server started on http://localhost:37655


Set the default backend. Note: Besides the local server, you may use also `OpenAI` or other API endpoints.

In [2]:
set_default_backend(RuntimeEndpoint(f"http://localhost:{port}"))

[2025-07-19 00:42:30] INFO:     127.0.0.1:58604 - "GET /get_model_info HTTP/1.1" 200 OK


## Basic Usage

The most simple way of using SGLang frontend language is a simple question answer dialog between a user and an assistant.

In [3]:
@function
def basic_qa(s, question):
    s += system(f"You are a helpful assistant than can answer questions.")
    s += user(question)
    s += assistant(gen("answer", max_tokens=512))

In [4]:
state = basic_qa("List 3 countries and their capitals.")
print_highlight(state["answer"])

[2025-07-19 00:42:30] Prefill batch. #new-seq: 1, #new-token: 31, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-19T00:42:30.532706


[2025-07-19 00:42:30] Decode batch. #running-req: 1, #token: 64, token usage: 0.00, cuda graph: False, gen throughput (token/s): 5.91, #queue-req: 0, timestamp: 2025-07-19T00:42:30.864960
[2025-07-19 00:42:30] INFO:     127.0.0.1:58618 - "POST /generate HTTP/1.1" 200 OK


## Multi-turn Dialog

SGLang frontend language can also be used to define multi-turn dialogs.

In [5]:
@function
def multi_turn_qa(s):
    s += system(f"You are a helpful assistant than can answer questions.")
    s += user("Please give me a list of 3 countries and their capitals.")
    s += assistant(gen("first_answer", max_tokens=512))
    s += user("Please give me another list of 3 countries and their capitals.")
    s += assistant(gen("second_answer", max_tokens=512))
    return s


state = multi_turn_qa()
print_highlight(state["first_answer"])
print_highlight(state["second_answer"])

[2025-07-19 00:42:30] Prefill batch. #new-seq: 1, #new-token: 18, #cached-token: 18, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-19T00:42:30.892654


[2025-07-19 00:42:31] Decode batch. #running-req: 1, #token: 74, token usage: 0.00, cuda graph: False, gen throughput (token/s): 105.62, #queue-req: 0, timestamp: 2025-07-19T00:42:31.243651
[2025-07-19 00:42:31] INFO:     127.0.0.1:58626 - "POST /generate HTTP/1.1" 200 OK


[2025-07-19 00:42:31] Prefill batch. #new-seq: 1, #new-token: 23, #cached-token: 74, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-19T00:42:31.254815


[2025-07-19 00:42:31] Decode batch. #running-req: 1, #token: 136, token usage: 0.01, cuda graph: False, gen throughput (token/s): 109.74, #queue-req: 0, timestamp: 2025-07-19T00:42:31.608136
[2025-07-19 00:42:31] INFO:     127.0.0.1:58642 - "POST /generate HTTP/1.1" 200 OK


## Control flow

You may use any Python code within the function to define more complex control flows.

In [6]:
@function
def tool_use(s, question):
    s += assistant(
        "To answer this question: "
        + question
        + ". I need to use a "
        + gen("tool", choices=["calculator", "search engine"])
        + ". "
    )

    if s["tool"] == "calculator":
        s += assistant("The math expression is: " + gen("expression"))
    elif s["tool"] == "search engine":
        s += assistant("The key word to search is: " + gen("word"))


state = tool_use("What is 2 * 2?")
print_highlight(state["tool"])
print_highlight(state["expression"])

[2025-07-19 00:42:31] Prefill batch. #new-seq: 1, #new-token: 25, #cached-token: 8, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-19T00:42:31.625479
[2025-07-19 00:42:31] INFO:     127.0.0.1:58644 - "POST /generate HTTP/1.1" 200 OK
[2025-07-19 00:42:31] Prefill batch. #new-seq: 1, #new-token: 2, #cached-token: 31, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-19T00:42:31.650072
[2025-07-19 00:42:31] Prefill batch. #new-seq: 1, #new-token: 3, #cached-token: 31, token usage: 0.00, #running-req: 1, #queue-req: 0, timestamp: 2025-07-19T00:42:31.650973
[2025-07-19 00:42:31] INFO:     127.0.0.1:58658 - "POST /generate HTTP/1.1" 200 OK


[2025-07-19 00:42:31] Prefill batch. #new-seq: 1, #new-token: 13, #cached-token: 33, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-19T00:42:31.685459


[2025-07-19 00:42:32] Decode batch. #running-req: 1, #token: 83, token usage: 0.00, cuda graph: False, gen throughput (token/s): 95.87, #queue-req: 0, timestamp: 2025-07-19T00:42:32.025378
[2025-07-19 00:42:32] INFO:     127.0.0.1:58666 - "POST /generate HTTP/1.1" 200 OK


## Parallelism

Use `fork` to launch parallel prompts. Because `sgl.gen` is non-blocking, the for loop below issues two generation calls in parallel.

In [7]:
@function
def tip_suggestion(s):
    s += assistant(
        "Here are two tips for staying healthy: "
        "1. Balanced Diet. 2. Regular Exercise.\n\n"
    )

    forks = s.fork(2)
    for i, f in enumerate(forks):
        f += assistant(
            f"Now, expand tip {i+1} into a paragraph:\n"
            + gen("detailed_tip", max_tokens=256, stop="\n\n")
        )

    s += assistant("Tip 1:" + forks[0]["detailed_tip"] + "\n")
    s += assistant("Tip 2:" + forks[1]["detailed_tip"] + "\n")
    s += assistant(
        "To summarize the above two tips, I can say:\n" + gen("summary", max_tokens=512)
    )


state = tip_suggestion()
print_highlight(state["summary"])

[2025-07-19 00:42:32] Prefill batch. #new-seq: 1, #new-token: 35, #cached-token: 14, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-19T00:42:32.130004
[2025-07-19 00:42:32] Prefill batch. #new-seq: 1, #new-token: 35, #cached-token: 14, token usage: 0.00, #running-req: 1, #queue-req: 0, timestamp: 2025-07-19T00:42:32.130935


[2025-07-19 00:42:32] Decode batch. #running-req: 2, #token: 112, token usage: 0.01, cuda graph: False, gen throughput (token/s): 128.39, #queue-req: 0, timestamp: 2025-07-19T00:42:32.555014


[2025-07-19 00:42:32] Decode batch. #running-req: 2, #token: 192, token usage: 0.01, cuda graph: False, gen throughput (token/s): 219.08, #queue-req: 0, timestamp: 2025-07-19T00:42:32.920174


[2025-07-19 00:42:33] Decode batch. #running-req: 2, #token: 272, token usage: 0.01, cuda graph: False, gen throughput (token/s): 219.75, #queue-req: 0, timestamp: 2025-07-19T00:42:33.284227


[2025-07-19 00:42:33] INFO:     127.0.0.1:58676 - "POST /generate HTTP/1.1" 200 OK
[2025-07-19 00:42:33] Decode batch. #running-req: 1, #token: 198, token usage: 0.01, cuda graph: False, gen throughput (token/s): 181.68, #queue-req: 0, timestamp: 2025-07-19T00:42:33.641998


[2025-07-19 00:42:33] INFO:     127.0.0.1:58692 - "POST /generate HTTP/1.1" 200 OK
[2025-07-19 00:42:33] Prefill batch. #new-seq: 1, #new-token: 338, #cached-token: 39, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-19T00:42:33.899833
[2025-07-19 00:42:34] Decode batch. #running-req: 1, #token: 388, token usage: 0.02, cuda graph: False, gen throughput (token/s): 105.90, #queue-req: 0, timestamp: 2025-07-19T00:42:34.019719


[2025-07-19 00:42:34] Decode batch. #running-req: 1, #token: 428, token usage: 0.02, cuda graph: False, gen throughput (token/s): 115.05, #queue-req: 0, timestamp: 2025-07-19T00:42:34.367394


[2025-07-19 00:42:34] Decode batch. #running-req: 1, #token: 468, token usage: 0.02, cuda graph: False, gen throughput (token/s): 115.18, #queue-req: 0, timestamp: 2025-07-19T00:42:34.714674


[2025-07-19 00:42:35] Decode batch. #running-req: 1, #token: 508, token usage: 0.02, cuda graph: False, gen throughput (token/s): 115.12, #queue-req: 0, timestamp: 2025-07-19T00:42:35.062148


[2025-07-19 00:42:35] Decode batch. #running-req: 1, #token: 548, token usage: 0.03, cuda graph: False, gen throughput (token/s): 111.43, #queue-req: 0, timestamp: 2025-07-19T00:42:35.421133
[2025-07-19 00:42:35] INFO:     127.0.0.1:58708 - "POST /generate HTTP/1.1" 200 OK


## Constrained Decoding

Use `regex` to specify a regular expression as a decoding constraint. This is only supported for local models.

In [8]:
@function
def regular_expression_gen(s):
    s += user("What is the IP address of the Google DNS servers?")
    s += assistant(
        gen(
            "answer",
            temperature=0,
            regex=r"((25[0-5]|2[0-4]\d|[01]?\d\d?).){3}(25[0-5]|2[0-4]\d|[01]?\d\d?)",
        )
    )


state = regular_expression_gen()
print_highlight(state["answer"])

[2025-07-19 00:42:35] Prefill batch. #new-seq: 1, #new-token: 18, #cached-token: 12, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-19T00:42:35.478875


[2025-07-19 00:42:36] INFO:     127.0.0.1:57736 - "POST /generate HTTP/1.1" 200 OK


Use `regex` to define a `JSON` decoding schema.

In [9]:
character_regex = (
    r"""\{\n"""
    + r"""    "name": "[\w\d\s]{1,16}",\n"""
    + r"""    "house": "(Gryffindor|Slytherin|Ravenclaw|Hufflepuff)",\n"""
    + r"""    "blood status": "(Pure-blood|Half-blood|Muggle-born)",\n"""
    + r"""    "occupation": "(student|teacher|auror|ministry of magic|death eater|order of the phoenix)",\n"""
    + r"""    "wand": \{\n"""
    + r"""        "wood": "[\w\d\s]{1,16}",\n"""
    + r"""        "core": "[\w\d\s]{1,16}",\n"""
    + r"""        "length": [0-9]{1,2}\.[0-9]{0,2}\n"""
    + r"""    \},\n"""
    + r"""    "alive": "(Alive|Deceased)",\n"""
    + r"""    "patronus": "[\w\d\s]{1,16}",\n"""
    + r"""    "bogart": "[\w\d\s]{1,16}"\n"""
    + r"""\}"""
)


@function
def character_gen(s, name):
    s += user(
        f"{name} is a character in Harry Potter. Please fill in the following information about this character."
    )
    s += assistant(gen("json_output", max_tokens=256, regex=character_regex))


state = character_gen("Harry Potter")
print_highlight(state["json_output"])

[2025-07-19 00:42:36] Prefill batch. #new-seq: 1, #new-token: 24, #cached-token: 14, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-19T00:42:36.556776


[2025-07-19 00:42:36] Decode batch. #running-req: 1, #token: 58, token usage: 0.00, cuda graph: False, gen throughput (token/s): 29.92, #queue-req: 0, timestamp: 2025-07-19T00:42:36.758222


[2025-07-19 00:42:37] Decode batch. #running-req: 1, #token: 98, token usage: 0.00, cuda graph: False, gen throughput (token/s): 110.33, #queue-req: 0, timestamp: 2025-07-19T00:42:37.120766


[2025-07-19 00:42:37] Decode batch. #running-req: 1, #token: 138, token usage: 0.01, cuda graph: False, gen throughput (token/s): 109.26, #queue-req: 0, timestamp: 2025-07-19T00:42:37.486869
[2025-07-19 00:42:37] INFO:     127.0.0.1:57746 - "POST /generate HTTP/1.1" 200 OK


## Batching 

Use `run_batch` to run a batch of prompts.

In [10]:
@function
def text_qa(s, question):
    s += user(question)
    s += assistant(gen("answer", stop="\n"))


states = text_qa.run_batch(
    [
        {"question": "What is the capital of the United Kingdom?"},
        {"question": "What is the capital of France?"},
        {"question": "What is the capital of Japan?"},
    ],
    progress_bar=True,
)

for i, state in enumerate(states):
    print_highlight(f"Answer {i+1}: {states[i]['answer']}")

[2025-07-19 00:42:37] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 13, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-19T00:42:37.639690
[2025-07-19 00:42:37] INFO:     127.0.0.1:57758 - "POST /generate HTTP/1.1" 200 OK


  0%|          | 0/3 [00:00<?, ?it/s]

[2025-07-19 00:42:37] Prefill batch. #new-seq: 1, #new-token: 11, #cached-token: 17, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-19T00:42:37.663600
[2025-07-19 00:42:37] Prefill batch. #new-seq: 1, #new-token: 9, #cached-token: 17, token usage: 0.00, #running-req: 1, #queue-req: 0, timestamp: 2025-07-19T00:42:37.664524


 33%|███▎      | 1/3 [00:00<00:00,  8.45it/s]

 67%|██████▋   | 2/3 [00:00<00:00, 13.39it/s]

100%|██████████| 3/3 [00:00<00:00, 22.07it/s]

[2025-07-19 00:42:37] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 19, token usage: 0.00, #running-req: 2, #queue-req: 0, timestamp: 2025-07-19T00:42:37.692457
[2025-07-19 00:42:37] INFO:     127.0.0.1:57768 - "POST /generate HTTP/1.1" 200 OK
[2025-07-19 00:42:37] INFO:     127.0.0.1:57774 - "POST /generate HTTP/1.1" 200 OK
[2025-07-19 00:42:37] INFO:     127.0.0.1:57760 - "POST /generate HTTP/1.1" 200 OK


## Streaming 

Use `stream` to stream the output to the user.

In [11]:
@function
def text_qa(s, question):
    s += user(question)
    s += assistant(gen("answer", stop="\n"))


state = text_qa.run(
    question="What is the capital of France?", temperature=0.1, stream=True
)

for out in state.text_iter():
    print(out, end="", flush=True)

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
What is the capital of France?<|im_end|>
<|im_start|>assistant


[2025-07-19 00:42:37] INFO:     127.0.0.1:57778 - "POST /generate HTTP/1.1" 200 OK
[2025-07-19 00:42:37] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 25, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-19T00:42:37.807607
The

 capital

 of

 France

 is

 Paris

.

<|im_end|>


## Complex Prompts

You may use `{system|user|assistant}_{begin|end}` to define complex prompts.

In [12]:
@function
def chat_example(s):
    s += system("You are a helpful assistant.")
    # Same as: s += s.system("You are a helpful assistant.")

    with s.user():
        s += "Question: What is the capital of France?"

    s += assistant_begin()
    s += "Answer: " + gen("answer", max_tokens=100, stop="\n")
    s += assistant_end()


state = chat_example()
print_highlight(state["answer"])

[2025-07-19 00:42:37] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 14, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-19T00:42:37.893892
[2025-07-19 00:42:37] Decode batch. #running-req: 1, #token: 37, token usage: 0.00, cuda graph: False, gen throughput (token/s): 118.26, #queue-req: 0, timestamp: 2025-07-19T00:42:37.960381
[2025-07-19 00:42:37] INFO:     127.0.0.1:57790 - "POST /generate HTTP/1.1" 200 OK


In [13]:
terminate_process(server_process)

[2025-07-19 00:42:37] Child process unexpectedly failed with exitcode=9. pid=55952


## Multi-modal Generation

You may use SGLang frontend language to define multi-modal prompts.
See [here](https://docs.sglang.ai/supported_models/generative_models.html) for supported models.

In [14]:
server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path Qwen/Qwen2.5-VL-7B-Instruct --host 0.0.0.0"
)

wait_for_server(f"http://localhost:{port}")
print(f"Server started on http://localhost:{port}")

[2025-07-19 00:42:48] server_args=ServerArgs(model_path='Qwen/Qwen2.5-VL-7B-Instruct', tokenizer_path='Qwen/Qwen2.5-VL-7B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Qwen/Qwen2.5-VL-7B-Instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, impl='auto', host='0.0.0.0', port=39569, nccl_port=None, mem_fraction_static=0.7835956249999999, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_se

You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.
[2025-07-19 00:42:50] You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


[2025-07-19 00:42:51] Inferred chat template from model path: qwen2-vl


You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.
[2025-07-19 00:43:00] You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


[2025-07-19 00:43:01] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-07-19 00:43:01] Init torch distributed begin.


...

[2025-07-19 00:43:10] INFO:     Started server process [56735]
[2025-07-19 00:43:10] INFO:     Waiting for application startup.
[2025-07-19 00:43:10] INFO:     Application startup complete.
[2025-07-19 00:43:10] INFO:     Uvicorn running on http://0.0.0.0:39569 (Press CTRL+C to quit)


[2025-07-19 00:43:11] INFO:     127.0.0.1:57774 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-19 00:43:11] INFO:     127.0.0.1:57788 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-19 00:43:11] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-19T00:43:11.723640


[2025-07-19 00:43:12] INFO:     127.0.0.1:57798 - "POST /generate HTTP/1.1" 200 OK
[2025-07-19 00:43:12] The server is fired up and ready to roll!


Server started on http://localhost:39569


In [15]:
set_default_backend(RuntimeEndpoint(f"http://localhost:{port}"))

[2025-07-19 00:43:16] INFO:     127.0.0.1:60732 - "GET /get_model_info HTTP/1.1" 200 OK


Ask a question about an image.

In [16]:
@function
def image_qa(s, image_file, question):
    s += user(image(image_file) + question)
    s += assistant(gen("answer", max_tokens=256))


image_url = "https://github.com/sgl-project/sglang/blob/main/test/lang/example_image.png?raw=true"
image_bytes, _ = load_image(image_url)
state = image_qa(image_bytes, "What is in the image?")
print_highlight(state["answer"])

[2025-07-19 00:43:17] Prefill batch. #new-seq: 1, #new-token: 307, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-19T00:43:17.316755


[2025-07-19 00:43:18] Decode batch. #running-req: 1, #token: 340, token usage: 0.02, cuda graph: False, gen throughput (token/s): 4.89, #queue-req: 0, timestamp: 2025-07-19T00:43:18.394460


[2025-07-19 00:43:19] Decode batch. #running-req: 1, #token: 380, token usage: 0.02, cuda graph: False, gen throughput (token/s): 60.47, #queue-req: 0, timestamp: 2025-07-19T00:43:19.055993
[2025-07-19 00:43:19] INFO:     127.0.0.1:60746 - "POST /generate HTTP/1.1" 200 OK


In [17]:
terminate_process(server_process)